In [1]:
import h5py
import os
import numpy as np
from tqdm import tqdm
import pickle

DATA_DIR = "../data/"
RAW_DIR = os.path.join(DATA_DIR, "raw")
PREP_DIR = os.path.join(DATA_DIR, "prep")
TRAIN_DIR = os.path.join(RAW_DIR,"train")

## Objective

    우리의 카카오 데이터를 한 데 모읍시다. 80기가가 되는데, 어떻게 줄여야 할지 고민해보아요

In [2]:
f = h5py.File(os.path.join(TRAIN_DIR,"train.chunk.01"),'r')

# attribute 별 memory사이즈를 계산
attr_size_dict = {}
for key in f['train'].keys():
    value = f['train'][key]
    mb_size = (value.size * value.dtype.itemsize) // (1024**2)
    attr_size_dict[key] = mb_size

tot_size = sum(attr_size_dict.values())

for key, value in attr_size_dict.items():
    print("{:10}의 메모리 size : {:4}MB({:2.2f}%)".format(
        key,value,value/tot_size*100))

bcateid   의 메모리 size :    3MB(0.03%)
brand     의 메모리 size :  124MB(1.38%)
dcateid   의 메모리 size :    3MB(0.03%)
img_feat  의 메모리 size : 7812MB(87.04%)
maker     의 메모리 size :  274MB(3.05%)
mcateid   의 메모리 size :    3MB(0.03%)
model     의 메모리 size :  142MB(1.58%)
pid       의 메모리 size :   11MB(0.12%)
price     의 메모리 size :    3MB(0.03%)
product   의 메모리 size :  583MB(6.50%)
scateid   의 메모리 size :    3MB(0.03%)
updttm    의 메모리 size :   14MB(0.16%)


Image Feature가 차지하는 메모리 분량이 87%나 된다.
즉 89기가 중 대략 77기가가 이미지 정보이고, 12기가 정도가 그 외 정보가 된다. 

이미지 Feature를 제외하고 다른 Feature들은 하나의 h5 파일로 모아보자

In [3]:
merge_dir = TRAIN_DIR
# merge 하고자 하는 데이터 셋의 파일 path 
file_paths = [os.path.join(merge_dir,file_name)
              for file_name in os.listdir(merge_dir)
              if "chunk" in file_name]

prep_path = os.path.join(PREP_DIR,"train_string.h5")
write_file = h5py.File(prep_path,"w")

attrs = list(f['train'].keys()) # Merge할 attribute의 list
attrs.remove('img_feat') # img_feat 제거
for attr in tqdm(attrs):
    values = []
    for file_path in file_paths:
        with h5py.File(file_path,'r') as read_file:
            value = read_file['train'][attr][:]
        values.append(value)
    merge_value = np.concatenate(values)
    write_file.create_dataset(attr,data=merge_value)
write_file.close()

print("저장된 파일의 사이즈 : {}mb".format(
    os.stat(prep_path).st_size // (1024**2)))

100%|██████████| 11/11 [00:35<00:00,  3.24s/it]

저장된 파일의 사이즈 : 9519mb


단일 파일로 이제 9기가 정도의 파일로 줄어들었다. 로컬 컴퓨터에서도 이제 데이터 탐색이 가능한 사이즈로 줄어들었다. 

이와 같은 짓거리를 Test데이터셋에서도 해야 하고, dev데이터셋에서도 해야한다

직접 위의 코드를 돌리고 싶으면
bash 창에서, 

```shell
python data_utils.py preprocess ./data/raw/train/ ./data/prep/train_string.h5
python data_utils.py preprocess ./data/raw/test/ ./data/prep/test_string.h5
python data_utils.py preprocess ./data/raw/dev/ ./data/prep/dev_string.h5
```

을 하면 각각 train, test, dev 데이터셋이 생긴다